# Story data 
In this notebook, we will sketch out a tale worth telling. It will be told in the ```story.ipynb``` notebook

The first order of buisiness is to get all the alpha-alpha and beta-beta excitations of ROHF. We will attempt to compare them to the ones in CUHF.


In [1]:
import psi4
import numpy as np
from scipy.linalg import eigh

In [19]:
psi4.set_options({"basis":"sto-3g", "reference": "rhf", "scf_type":"pk", "guess_mix":"True"})
allyl = psi4.geometry("""
0 1
O
H 1 1.1
H 1 1.1 2 104
symmetry c1

""")
E, wfn = psi4.energy("scf", return_wfn = True)
E

-74.9420798986689

In [40]:
def displayCISHamiltonian(wfn, geom):
    """displays the CIS hamiltonian in MO basis"""
    # getting the orbital energies
    
    epsilon_a, C_a = wfn.epsilon_a().np, wfn.Ca().np
    epsilon_b, C_b = wfn.epsilon_b().np, wfn.Cb().np
    epsilon = np.append(epsilon_a, epsilon_b) # accounts for the fact that the energies might be different
    sortedorder = np.argsort(epsilon)
    epsilon = np.sort(epsilon)
    
    # make the C matrix => it contains all the orbitals, both alpha and beta
    C = np.block([[C_a, np.zeros(C_a.shape)], [np.zeros(C_b.shape), C_b]]) # accounts for the fact that the orbitals might be different (uhf, cuhf)
    C = C[:, sortedorder] # sort the eigenfunctions
    
    basis = wfn.basisset()
    mints = psi4.core.MintsHelper(basis)

    # getting the two electron integrals in correct basis => we need it in MO basis
    tei = mints.ao_eri().np # given in chemists notation

    #change the basis of the tei
    tei_int = np.kron(np.eye(2), tei)
    tei_big = np.kron(np.eye(2), tei_int.T)

    tei_ao = tei_big.transpose(0, 2, 1, 3) - tei_big.transpose(0, 2, 3, 1) # accounts for both coulomb and exchange, switch to physisists notation
    tei_mo = np.einsum("pQRS,pP->PQRS", np.einsum("pqRS,qQ->pQRS", np.einsum("pqrS,rR->pqRS", np.einsum("pqrs,sS->pqrS", tei_ao, C, optimize=True), C, optimize=True), C, optimize=True), C, optimize=True)
    
    occupied_a = wfn.nalpha()
    occupied_b = wfn.nbeta()
    available = mints.nbf()
    virtual_a = available - occupied_a
    virtual_b = available - occupied_b
    #getting the excitations
    excitations_a, excitations_b = [], []
    for orbital in range(occupied_a): 
        for another_orbital in range(occupied_a, available):
            excitations_a.append((orbital, another_orbital))
    
    for orbital in range(occupied_b): 
        for another_orbital in range(occupied_b, available):
            excitations_b.append((orbital, another_orbital))
    
    # getting the hamiltonian, we will do it in two parts here
    dim_a = occupied_a*virtual_a
    dim_b = occupied_b*virtual_b
    H_cis_a = np.zeros((dim_a + dim_b, dim_a + dim_b))
    for row, excitation in enumerate(excitations_a):
        i, a = excitation
        i *= 2
        a *= 2
        for collumn, another_excitation in enumerate(excitations_a):
            j, b = another_excitation
            j *= 2
            b *= 2   
            H_cis_a[row, collumn] = (epsilon[a] - epsilon[i])*(i == j)*(a == b) + tei_mo[a, j, i, b]
    
    for row, excitation in enumerate(excitations_b):
        i, a = excitation
        i *= 2 + 1
        a *= 2 + 1
        for collumn, another_excitation in enumerate(excitations_b):
            j, b = another_excitation
            j *= 2 + 1
            b *= 2 + 1
            H_cis_a[dima_a + row, dim_a + collumn] = (epsilon[a] - epsilon[i])*(i == j)*(a == b) + tei_mo[a, j, i, b]

    # get the E_0 value in the hamiltonian
    extra_row = np.zeros((H_cis_a.shape[1],))
    H_cis_a = np.vstack((extra_row, H_cis_a))
    extra_collumn = np.zeros((H_cis_a.shape[0], 1))
    H_cis_a = np.hstack((extra_collumn, H_cis_a))
    H_cis_a[0,0] = E
    return H_cis_a, excitations_a, excitations_b
ham, excitations_a, excitations_b = displayCISHamiltonian(wfn, allyl)

In [41]:
from scipy.linalg import eigh
exits, coefs = eigh(ham)

In [43]:
def GetExitations(exits, coefs, excitations, filepath="NoNameGiven"):
        """Get the excitation energies and the contributions"""
        if filepath == "NoNameGiven":
            raise ValueError("no path specified")
        from pathlib import Path
        Path(f"{filepath}").touch()
        datafile = open(f"{filepath}", "w")
        contrib = coefs[1:, 1:]**2
        energies = exits[1:]
        counterdict = {} # added to check how many times a certain excitation occurs
        datafile.writelines(f"scf energy for molecule: {E}\n")
        for state, energy in enumerate(energies):
            datafile.writelines(f" {state + 1} : {energy}\n")
            for excitation, contribution in enumerate(contrib[:, state]):
                if contribution*100 > 1:
                    datafile.writelines(f"\t{contribution:.3f} : {excitations[excitation]}\n")
                    if excitations[excitation] not in counterdict:
                        counterdict[excitations[excitation]] = 0
                    counterdict[excitations[excitation]] += 1   
        datafile.close()
GetExitations(exits, coefs, excitations_a, "/user/gent/440/vsc44013/Bachelor_project/Bachelor_project/definitive_versions/theory/outline_BachelorProject/code/testdata_a.txt")

In [44]:
def GetExitations(exits, coefs, excitations, filepath="NoNameGiven"):
        """Get the excitation energies and the contributions"""
        if filepath == "NoNameGiven":
            raise ValueError("no path specified")
        from pathlib import Path
        Path(f"{filepath}").touch()
        datafile = open(f"{filepath}", "w")
        contrib = coefs[1:, 1:]**2
        energies = exits[1:]
        counterdict = {} # added to check how many times a certain excitation occurs
        datafile.writelines(f"scf energy for molecule: {E}\n")
        for state, energy in enumerate(energies):
            datafile.writelines(f" {state + 1} : {energy}\n")
            for excitation, contribution in enumerate(contrib[:, state]):
                if contribution*100 > 1:
                    datafile.writelines(f"\t{contribution:.3f} : {excitations[excitation]}\n")
                    if excitations[excitation] not in counterdict:
                        counterdict[excitations[excitation]] = 0
                    counterdict[excitations[excitation]] += 1   
        datafile.close()
GetExitations(exits, coefs, excitations_b, "/user/gent/440/vsc44013/Bachelor_project/Bachelor_project/definitive_versions/theory/outline_BachelorProject/code/testdata_b.txt")